# Reading Model Predictions and Bet365 Odds

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
from rapidfuzz import process
from datetime import datetime

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

csv_url = "https://docs.google.com/spreadsheets/d/1WfEG-1icUjj6k7TGePJQEXH-w0TLEIcN/export?format=csv"
uefa = pd.read_csv(csv_url, dtype={'FTHG': 'Int64', 'FTAG': 'Int64', 'HTHG': 'Int64', 'HTAG': 'Int64'})
uefa['FT'] = uefa['FTHG'].astype(str) + ' - ' + uefa['FTAG'].astype(str)
uefa['HT'] = '(' + uefa['HTHG'].astype(str) + '-' + uefa['HTAG'].astype(str) + ')'
uefa['FTTG'] = uefa['FTHG'] + uefa['FTAG']
uefa['HTTG'] = uefa['HTHG'] + uefa['HTAG']

predictions = pd.read_excel('C:/Users/99451/Desktop/MODEL/2025/dixon_coles_model_predictions/_predictions.xlsx')
bet365_odds = pd.read_excel('C:/Users/99451/Desktop/MODEL/2025/dixon_coles_model_predictions/final_odds.xlsx')

print(f"Games found: {len(predictions)} in predictions and {len(bet365_odds)} in odds dataset.")
bet365_odds.tail()

Games found: 5727 in predictions and 5886 in odds dataset.


,Home,Away,FT1,FTX,FT2,DC1X,DC12,DCX2,HT1,HTX,HT2,HT1X,HT12,HTX2,BTTS,OTTS,1.5O,1.5U,2.5O,2.5U,3.5O,3.5U,4.5O,4.5U,HT0.5O,HT0.5U,HT1.5O,HT1.5U
5881,Banfield,Huracan,3.10,2.90,2.50,1.50,1.40,1.36,4.33,1.8,3.50,1.29,1.91,1.20,2.38,1.53,1.67,2.10,3.10,1.36,7.00,1.10,17.00,1.03,1.67,2.10,4.50,1.18
5882,San Lorenzo,Lanus,2.50,3.00,3.10,1.36,1.36,1.50,3.50,1.8,4.33,1.20,1.91,1.29,2.50,1.50,1.67,2.10,3.40,1.33,7.00,1.10,17.00,1.03,1.73,2.08,4.50,1.18
5883,Fredrikstad,Rosenborg,2.70,3.25,2.63,1.44,1.33,1.44,3.25,2.1,3.25,1.33,1.67,1.33,1.80,1.95,1.33,3.25,2.05,1.75,3.75,1.25,8.00,1.08,1.40,2.75,3.00,1.36
5884,Betis,Chelsea,4.20,3.60,1.85,1.91,1.29,1.22,4.50,2.3,2.38,1.55,1.62,1.20,1.67,2.10,1.22,4.00,1.75,2.05,2.75,1.40,5.50,1.14,1.33,3.25,2.63,1.44
5885,Bodo/Glimt,Viking,1.45,5.00,5.75,1.15,1.18,2.63,1.83,2.8,5.00,1.14,1.40,1.83,1.44,2.63,1.08,8.00,1.33,3.40,1.80,1.91,2.75,1.40,1.18,4.50,1.90,1.90


# Merging 2 DataFrames for similarity of values

In [2]:
# Use predictions key values as the canonical list
home_keys = predictions['Home'].unique().tolist()
away_keys = predictions['Away'].unique().tolist()

def get_canonical(val, canonical_list, threshold=85):
    """
    For a given value from df2, find the best matching canonical value from df1
    using a fuzzy matching score. If no match meets the threshold, return the original value.
    """
    match = process.extractOne(val, canonical_list, score_cutoff=threshold)
    if match:
        return match[0]
    return val

# Replace values in bet365_odds Home and Away columns using the canonical mapping from predictions
bet365_odds['Home'] = bet365_odds['Home'].apply(lambda x: get_canonical(x, home_keys))
bet365_odds['Away'] = bet365_odds['Away'].apply(lambda x: get_canonical(x, away_keys))

# Merge 2 dataframes on Home and Away columns
merged_df = pd.merge(predictions, bet365_odds, on=['Home', 'Away'], how='inner')
merged_df = merged_df.drop_duplicates(subset=['League','Home', 'Away'], keep='last')
merged_df.tail()

,League,Home,Away,FT1_x,FTX_x,FT2_x,FTR,DC1X_x,DC12_x,DCX2_x,1.5O_x,2.5O_x,3.5U_x,4.5U_x,BTTS_x,HT1_x,HTX_x,HT2_x,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O_x,HT1.5U_x,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U,FT1_y,FTX_y,FT2_y,DC1X_y,DC12_y,DCX2_y,HT1_y,HTX_y,HT2_y,HT1X,HT12,HTX2,BTTS_y,OTTS,1.5O_y,1.5U,2.5O_y,2.5U,3.5O,3.5U_y,4.5O,4.5U_y,HT0.5O_y,HT0.5U,HT1.5O,HT1.5U_y
4980,Argentina,Independiente,Godoy Cruz,70.94,22.63,6.38,2-0,93.57,77.32,29.01,79.53,53.10,69.11,84.71,43.38,49.07,38.91,11.72,0-0,87.98,60.79,50.63,71.29,61.97,88.75,45.92,64.23,12.68,62.57,97.49,1.80,3.00,5.50,1.14,1.36,1.95,2.60,1.91,6.00,1.11,1.80,1.50,2.50,1.50,1.57,2.25,2.87,1.37,6.00,1.13,15.00,1.03,1.62,2.20,4.0,1.22
4981,Greece,Panaitolikos,Levadiakos,35.08,34.61,30.32,0-0,69.69,65.40,64.93,47.55,21.92,91.98,97.58,30.22,20.00,59.27,20.73,0-0,79.27,40.73,80.00,48.38,83.24,57.11,53.38,20.80,17.81,94.56,95.77,2.20,3.10,3.50,1.29,1.36,1.67,3.10,1.83,4.50,1.17,1.83,1.36,2.25,1.57,1.57,2.25,2.88,1.40,6.00,1.13,15.00,1.03,1.62,2.20,4.0,1.22
4982,Switzerland,Sion,Yverdon,54.91,24.30,20.77,1-1,79.21,75.68,45.07,76.33,51.72,70.42,85.62,52.38,47.86,38.43,13.43,0-0,86.29,61.29,51.86,71.87,62.29,82.69,62.84,52.34,26.06,74.28,92.14,1.48,4.50,6.25,1.13,1.18,2.63,2.00,2.40,6.00,1.11,1.53,1.73,1.83,1.83,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.5,1.50
4983,Norway,Fredrikstad,Rosenborg,36.70,37.13,26.17,0-0,73.83,62.87,63.30,40.90,16.97,94.48,98.52,24.92,23.88,50.62,25.47,0-0,74.50,49.35,76.09,60.88,72.13,54.82,46.13,18.92,12.80,95.34,97.50,2.70,3.25,2.63,1.44,1.33,1.44,3.25,2.10,3.25,1.33,1.67,1.33,1.80,1.95,1.33,3.25,2.05,1.75,3.75,1.25,8.00,1.08,1.40,2.75,3.0,1.36
4984,Norway,Bodo/Glimt,Viking,72.20,13.90,13.42,3-1,86.10,85.62,27.32,93.02,81.62,35.05,54.13,70.08,73.46,25.19,0.00,1-0,98.65,73.46,25.19,73.46,59.93,95.06,73.52,81.19,38.62,39.91,84.32,1.45,5.00,5.75,1.15,1.18,2.63,1.83,2.80,5.00,1.14,1.40,1.83,1.44,2.63,1.08,8.00,1.33,3.40,1.80,1.91,2.75,1.40,1.18,4.50,1.9,1.90


# Scraping SoccerStats For Match Results

In [3]:
final =  pd.DataFrame()
liqa = ''
unique_leagues = predictions['League'].unique().tolist()

# Convert to lowercase and exclude UEFA competitions
uefa_list = ['unl', 'uel', 'ucl', 'ufcl']
list_2024 = ['norway_2024', 'sweden_2024', 'usa_2024']
unique_leagues = list(set([league.lower() for league in unique_leagues if league.lower() not in uefa_list]))

# Append list_2024 to unique_leagues
unique_leagues.extend(list_2024)

for i in unique_leagues:
    URL = "https://www.soccerstats.com/results.asp?league=" + i + "&pmtype=bydate"
    page = requests.get(URL)
    liqa = i
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="btable")
    sth = results.find_all("tr", class_="odd")
    sth


    date, league, home, away, ft, ht = [], [], [], [], [],[]
    for i in sth:
        date.append(i.find_all("td", align = 'right')[0].get_text(strip=True))
        league.append(liqa.capitalize())
        home.append(i.find_all("td", align = 'right')[1].get_text(strip=True))
        away.append(i.find("td", align = "left").get_text(strip = True))
        ft.append(i.find_all("td", align = 'center')[0].get_text(strip = True))
        try:
            ht.append(i.find_all("td", align = 'center')[2].get_text(strip = True))
        except IndexError as e:
            ht.append('NA')#print("Last output before error occurred:", i.find_all("td", align = 'center'))

    data = {'Date': date, 'League': league,'Home': home, 'Away': away, 'FT': ft, 'HT': ht}

# Create a DataFrame from the dictionary
    df = pd.DataFrame(data)

# Replace empty strings with NaN
    #next_df = df[(df['Date'] == formatted_date) & (df['HT'] == '')]
    df.replace('', pd.NA, inplace=True)

# Drop rows with NaN values
    df_cleaned = df.dropna()

#For Half-Time Results
    hthg, htag = [], []
    for i in df_cleaned['HT']:
        if i == 'NA':
            hthg.append('NA')
            htag.append('NA')
        elif i == '+' or i == '-':
            hthg.append('NA')
            htag.append('NA')
        else:
            try:
                hthg.append(int(i[1]))
                htag.append(int(i[3]))
            except IndexError as e:
                print("Last output before error occurred:", i)



#For Full-Time Results
    hg, ag, tg = [], [], []
    for i in df_cleaned['FT']:
        if len(i) < 5 or ':' in i:
            hg.append('NA')
            ag.append('NA')
            tg.append('NA')
        else:
            try:
                hghg = int(i.split(' - ')[0])
                hg.append(hghg)
                agag = int(i.split(' - ')[1])
                ag.append(agag)
                tg.append(hghg + agag)
            except:
                print(hghg + agag)

    
    df_cleaned['FTHG'], df_cleaned['FTAG'], df_cleaned['FTTG'] = hg, ag, tg
    df_cleaned['HTHG'], df_cleaned['HTAG'] = hthg, htag
    df_cleaned['HTTG'] = df_cleaned['HTHG'] + df_cleaned['HTAG']
    
    final = pd.concat([final, df_cleaned], ignore_index=True)
    
final = final[final['HT'] != 'NA']

# Get the date for today
today = datetime.now()

# Append the correct year based on if the date already occurred in 2025
def assign_year(date_str):
    if pd.isna(date_str):  # Handle NaN values
        return None
    
    # Check if the same day and month already occurred in 2025
    date_in_2025 = datetime.strptime(date_str + ' 2025', "%d %b %Y")

    # If this day and month already occurred in 2025, append 2025, otherwise 2024
    if date_in_2025 <= today:
        return f"{date_str} 2025"
    else:
        return f"{date_str} 2024"

# Example: Fix the 'date' column by removing the weekday and extra punctuation
final['Date'] = final['Date'].str.extract(r'(\d{1,2} \w{3})')  # Extract day and month part
final['Date'] = final['Date'].apply(assign_year)

# Convert to datetime format
final['Date'] = pd.to_datetime(final['Date'], format='%d %b %Y', errors='coerce')

# Filter rows before September 17th, 2024
final_filtered = final[final['Date'] >= pd.Timestamp('2024-09-17')]

# Remove "_YYYY" (4-digit year) at the end of usa, norway and sweden but keep other numbers
final_filtered['League'] = final_filtered['League'].str.replace(r'_\d{4}$', '', regex=True)

# Align columns of uefa to match final_filtered
uefa = uefa[final_filtered.columns]

# Concatenate
final_filtered = pd.concat([uefa, final_filtered], ignore_index=True)
combined = pd.concat([final_filtered.head(), final_filtered.tail()])
combined

,Date,League,Home,Away,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
0,9/17/2024,UCL,Juventus,PSV,3 - 1,(2-0),3,1,4,2,0,2
1,9/17/2024,UCL,Young Boys,Aston Villa,0 - 3,(0-2),0,3,3,0,2,2
2,9/17/2024,UCL,Bayern,Dinamo Zagreb,9 - 2,(3-0),9,2,11,3,0,3
3,9/17/2024,UCL,Milan,Liverpool,1 - 3,(1-2),1,3,4,1,2,3
4,9/17/2024,UCL,Real Madrid,Stuttgart,3 - 1,(0-0),3,1,4,0,0,0
11697,2024-10-20 00:00:00,Usa,Houston Dynamo,LA Galaxy,2 - 1,(1-0),2,1,3,1,0,1
11698,2024-10-20 00:00:00,Usa,Los Angeles FC,SJ Earthquakes,3 - 1,(0-1),3,1,4,0,1,1
11699,2024-10-20 00:00:00,Usa,Minnesota Utd,St. Louis City,4 - 1,(1-0),4,1,5,1,0,1
11700,2024-10-20 00:00:00,Usa,Real Salt Lake,Vancouver,2 - 1,(0-0),2,1,3,0,0,0
11701,2024-10-20 00:00:00,Usa,Seattle,Portland,1 - 1,(1-0),1,1,2,1,0,1


# Merging with Predictions + Odds Dataframes

In [4]:
# Merge on multiple columns
final_df = pd.merge(merged_df, final_filtered, on=['Home', 'Away'], how='inner')

# Drop duplicates based on the 'Home' and 'Away' columns
final_df_unique = final_df[~final_df.duplicated(subset=['League_x', 'Home', 'Away'], keep='last')]
final_df_unique = final_df_unique.dropna()

# Reset the index afterward
final_df_unique.reset_index(drop=True, inplace=True)

print('Number of games matched: ', len(final_df_unique))
final_df_unique.tail()

Number of games matched:  4474


,League_x,Home,Away,FT1_x,FTX_x,FT2_x,FTR,DC1X_x,DC12_x,DCX2_x,1.5O_x,2.5O_x,3.5U_x,4.5U_x,BTTS_x,HT1_x,HTX_x,HT2_x,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O_x,HT1.5U_x,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U,FT1_y,FTX_y,FT2_y,DC1X_y,DC12_y,DCX2_y,HT1_y,HTX_y,HT2_y,HT1X,HT12,HTX2,BTTS_y,OTTS,1.5O_y,1.5U,2.5O_y,2.5U,3.5O,3.5U_y,4.5O,4.5U_y,HT0.5O_y,HT0.5U,HT1.5O,HT1.5U_y,Date,League_y,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG
4469,Turkey,Konyaspor,Gaziantep,38.80,30.06,31.14,1-1,68.86,69.94,61.20,73.53,46.36,75.14,88.73,52.98,42.77,35.91,21.09,0-0,78.68,63.86,57.00,75.83,59.75,73.95,69.38,38.91,33.15,84.66,88.30,2.05,3.25,3.40,1.30,1.33,1.73,2.75,2.20,4.0,1.22,1.62,1.40,1.73,2.00,1.25,3.75,1.93,1.93,3.25,1.33,6.50,1.11,1.40,2.75,2.75,1.40,2025-03-28 00:00:00,Turkey,1 - 0,(0-0),1,0,1,0,0,0
4470,Argentina,Independiente,Godoy Cruz,70.94,22.63,6.38,2-0,93.57,77.32,29.01,79.53,53.10,69.11,84.71,43.38,49.07,38.91,11.72,0-0,87.98,60.79,50.63,71.29,61.97,88.75,45.92,64.23,12.68,62.57,97.49,1.80,3.00,5.50,1.14,1.36,1.95,2.60,1.91,6.0,1.11,1.80,1.50,2.50,1.50,1.57,2.25,2.87,1.37,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Argentina,4 - 0,(4-0),4,0,4,4,0,4
4471,Greece,Panaitolikos,Levadiakos,35.08,34.61,30.32,0-0,69.69,65.40,64.93,47.55,21.92,91.98,97.58,30.22,20.00,59.27,20.73,0-0,79.27,40.73,80.00,48.38,83.24,57.11,53.38,20.80,17.81,94.56,95.77,2.20,3.10,3.50,1.29,1.36,1.67,3.10,1.83,4.5,1.17,1.83,1.36,2.25,1.57,1.57,2.25,2.88,1.40,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Greece,1 - 2,(1-0),1,2,3,1,0,1
4472,Switzerland,Sion,Yverdon,54.91,24.30,20.77,1-1,79.21,75.68,45.07,76.33,51.72,70.42,85.62,52.38,47.86,38.43,13.43,0-0,86.29,61.29,51.86,71.87,62.29,82.69,62.84,52.34,26.06,74.28,92.14,1.48,4.50,6.25,1.13,1.18,2.63,2.00,2.40,6.0,1.11,1.53,1.73,1.83,1.83,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.50,1.50,2025-03-29 00:00:00,Switzerland,1 - 1,(0-1),1,1,2,0,1,1
4473,Norway,Bodo/Glimt,Viking,72.20,13.90,13.42,3-1,86.10,85.62,27.32,93.02,81.62,35.05,54.13,70.08,73.46,25.19,0.00,1-0,98.65,73.46,25.19,73.46,59.93,95.06,73.52,81.19,38.62,39.91,84.32,1.45,5.00,5.75,1.15,1.18,2.63,1.83,2.80,5.0,1.14,1.40,1.83,1.44,2.63,1.08,8.00,1.33,3.40,1.80,1.91,2.75,1.40,1.18,4.50,1.90,1.90,2025-04-06 00:00:00,Norway,1 - 0,(0-0),1,0,1,0,0,0


# Creating Results Columns

In [5]:
import numpy as np

# Add the new columns based on the condition
final_df_unique['FT1'] = np.where(final_df_unique['FTHG'] > final_df_unique['FTAG'], 1, 0)
final_df_unique['FTX'] = np.where(final_df_unique['FTHG'] == final_df_unique['FTAG'], 1, 0)
final_df_unique['FT2'] = np.where(final_df_unique['FTHG'] < final_df_unique['FTAG'], 1, 0)

final_df_unique['FT1X'] = np.where(final_df_unique['FTHG'] >= final_df_unique['FTAG'], 1, 0)
final_df_unique['FT12'] = np.where(final_df_unique['FTX'] == 0, 1, 0)
final_df_unique['FTX2'] = np.where(final_df_unique['FTHG'] <= final_df_unique['FTAG'], 1, 0)

final_df_unique['1.5O'] = np.where(final_df_unique['FTTG'] > 1.5, 1, 0)
final_df_unique['1.5U_y'] = np.where(final_df_unique['FTTG'] < 1.5, 1, 0)
final_df_unique['2.5O'] = np.where(final_df_unique['FTTG'] > 2.5, 1, 0)
final_df_unique['2.5U_y'] = np.where(final_df_unique['FTTG'] < 2.5, 1, 0)
final_df_unique['3.5O_y'] = np.where(final_df_unique['FTTG'] > 3.5, 1, 0)
final_df_unique['3.5U'] = np.where(final_df_unique['FTTG'] < 3.5, 1, 0)
final_df_unique['4.5O_y'] = np.where(final_df_unique['FTTG'] > 4.5, 1, 0)
final_df_unique['4.5U'] = np.where(final_df_unique['FTTG'] < 4.5, 1, 0)

final_df_unique['BTTS'] = np.where((final_df_unique['FTHG'] != 0) & (final_df_unique['FTAG'] != 0), 1, 0)
final_df_unique['OTTS_y'] = np.where(final_df_unique['BTTS'] == 0, 1, 0)

final_df_unique['HT1'] = np.where(final_df_unique['HTHG'] > final_df_unique['HTAG'], 1, 0)
final_df_unique['HTX'] = np.where(final_df_unique['HTHG'] == final_df_unique['HTAG'], 1, 0)
final_df_unique['HT2'] = np.where(final_df_unique['HTHG'] < final_df_unique['HTAG'], 1, 0)

final_df_unique['HT1X_y'] = np.where(final_df_unique['HTHG'] >= final_df_unique['HTAG'], 1, 0)
final_df_unique['HT12_y'] = np.where(final_df_unique['HTX'] == 0, 1, 0)
final_df_unique['HTX2_y'] = np.where(final_df_unique['HTHG'] <= final_df_unique['HTAG'], 1, 0)

final_df_unique['HT0.5O'] = np.where(final_df_unique['HTTG'] > 0.5, 1, 0)
final_df_unique['HT0.5U_y'] = np.where(final_df_unique['HTTG'] < 0.5, 1, 0)
final_df_unique['HT1.5O_y'] = np.where(final_df_unique['HTTG'] > 1.5, 1, 0)
final_df_unique['HT1.5U'] = np.where(final_df_unique['HTTG'] < 1.5, 1, 0)

print('Games Found: ', len(final_df_unique))
final_df_unique.tail()

Games Found:  4474


,League_x,Home,Away,FT1_x,FTX_x,FT2_x,FTR,DC1X_x,DC12_x,DCX2_x,1.5O_x,2.5O_x,3.5U_x,4.5U_x,BTTS_x,HT1_x,HTX_x,HT2_x,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O_x,HT1.5U_x,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U,FT1_y,FTX_y,FT2_y,DC1X_y,DC12_y,DCX2_y,HT1_y,HTX_y,HT2_y,HT1X,HT12,HTX2,BTTS_y,OTTS,1.5O_y,1.5U,2.5O_y,2.5U,3.5O,3.5U_y,4.5O,4.5U_y,HT0.5O_y,HT0.5U,HT1.5O,HT1.5U_y,Date,League_y,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,FT1,FTX,FT2,FT1X,FT12,FTX2,1.5O,1.5U_y,2.5O,2.5U_y,3.5O_y,3.5U,4.5O_y,4.5U,BTTS,OTTS_y,HT1,HTX,HT2,HT1X_y,HT12_y,HTX2_y,HT0.5O,HT0.5U_y,HT1.5O_y,HT1.5U
4469,Turkey,Konyaspor,Gaziantep,38.80,30.06,31.14,1-1,68.86,69.94,61.20,73.53,46.36,75.14,88.73,52.98,42.77,35.91,21.09,0-0,78.68,63.86,57.00,75.83,59.75,73.95,69.38,38.91,33.15,84.66,88.30,2.05,3.25,3.40,1.30,1.33,1.73,2.75,2.20,4.0,1.22,1.62,1.40,1.73,2.00,1.25,3.75,1.93,1.93,3.25,1.33,6.50,1.11,1.40,2.75,2.75,1.40,2025-03-28 00:00:00,Turkey,1 - 0,(0-0),1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
4470,Argentina,Independiente,Godoy Cruz,70.94,22.63,6.38,2-0,93.57,77.32,29.01,79.53,53.10,69.11,84.71,43.38,49.07,38.91,11.72,0-0,87.98,60.79,50.63,71.29,61.97,88.75,45.92,64.23,12.68,62.57,97.49,1.80,3.00,5.50,1.14,1.36,1.95,2.60,1.91,6.0,1.11,1.80,1.50,2.50,1.50,1.57,2.25,2.87,1.37,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Argentina,4 - 0,(4-0),4,0,4,4,0,4,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,1,0,1,0,1,0
4471,Greece,Panaitolikos,Levadiakos,35.08,34.61,30.32,0-0,69.69,65.40,64.93,47.55,21.92,91.98,97.58,30.22,20.00,59.27,20.73,0-0,79.27,40.73,80.00,48.38,83.24,57.11,53.38,20.80,17.81,94.56,95.77,2.20,3.10,3.50,1.29,1.36,1.67,3.10,1.83,4.5,1.17,1.83,1.36,2.25,1.57,1.57,2.25,2.88,1.40,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Greece,1 - 2,(1-0),1,2,3,1,0,1,0,0,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,1,0,1,0,0,1
4472,Switzerland,Sion,Yverdon,54.91,24.30,20.77,1-1,79.21,75.68,45.07,76.33,51.72,70.42,85.62,52.38,47.86,38.43,13.43,0-0,86.29,61.29,51.86,71.87,62.29,82.69,62.84,52.34,26.06,74.28,92.14,1.48,4.50,6.25,1.13,1.18,2.63,2.00,2.40,6.0,1.11,1.53,1.73,1.83,1.83,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.50,1.50,2025-03-29 00:00:00,Switzerland,1 - 1,(0-1),1,1,2,0,1,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,0,1
4473,Norway,Bodo/Glimt,Viking,72.20,13.90,13.42,3-1,86.10,85.62,27.32,93.02,81.62,35.05,54.13,70.08,73.46,25.19,0.00,1-0,98.65,73.46,25.19,73.46,59.93,95.06,73.52,81.19,38.62,39.91,84.32,1.45,5.00,5.75,1.15,1.18,2.63,1.83,2.80,5.0,1.14,1.40,1.83,1.44,2.63,1.08,8.00,1.33,3.40,1.80,1.91,2.75,1.40,1.18,4.50,1.90,1.90,2025-04-06 00:00:00,Norway,1 - 0,(0-0),1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1


# Creating Profit Columns for Initial Model Predictions

In [6]:
ft1p, ftxp, ft2p, ft1xp, ft12p, ftx2p = [], [], [], [], [], []
over15, under15, over25, under25, over35, under35, over45, under45 = [], [], [], [], [], [], [], []
btts, otts, ht1p, htxp, ht2p, ht1xp, ht12p, htx2p = [], [], [], [], [], [], [], []
htover05, htunder05, htover15, htunder15 = [], [], [], []

#if prediction == result -> coefficient, elif prediction != result -> 0, else -> -1
for i in range(len(final_df_unique)):
    ft_list = [final_df_unique['FT1_x'].iloc[i], final_df_unique['FTX_x'].iloc[i], final_df_unique['FT2_x'].iloc[i]]
    if (ft_list.index(max(ft_list)) == 0) and (final_df_unique['FT1'].iloc[i] == 1):
        ft1p.append(final_df_unique['FT1_y'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 0) and (final_df_unique['FT1'].iloc[i] != 1):
        ft1p.append(0)
    else:
        ft1p.append(-1)
    
    if (ft_list.index(max(ft_list)) == 1) and (final_df_unique['FTX'].iloc[i] == 1):
        ftxp.append(final_df_unique['FTX_y'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 1) and (final_df_unique['FTX'].iloc[i] != 1):
        ftxp.append(0)
    else:
        ftxp.append(-1)
    
    if (ft_list.index(max(ft_list)) == 2) and (final_df_unique['FT2'].iloc[i] == 1):
        ft2p.append(final_df_unique['FT2_y'].iloc[i])
    elif (ft_list.index(max(ft_list)) == 2) and (final_df_unique['FT2'].iloc[i] != 1):
        ft2p.append(0)
    else:
        ft2p.append(-1)

    dc_list = [final_df_unique['DC1X_x'].iloc[i], final_df_unique['DC12_x'].iloc[i], final_df_unique['DCX2_x'].iloc[i]]
    if (dc_list.index(max(dc_list)) == 0) and (final_df_unique['FT1X'].iloc[i] == 1):
        ft1xp.append(final_df_unique['DC1X_y'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 0) and (final_df_unique['FT1X'].iloc[i] != 1):
        ft1xp.append(0)
    else:
        ft1xp.append(-1)
    
    if (dc_list.index(max(dc_list)) == 1) and (final_df_unique['FT12'].iloc[i] == 1):
        ft12p.append(final_df_unique['DC12_y'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 1) and (final_df_unique['FT12'].iloc[i] != 1):
        ft12p.append(0)
    else:
        ft12p.append(-1)
    
    if (dc_list.index(max(dc_list)) == 2) and (final_df_unique['FTX2'].iloc[i] == 1):
        ftx2p.append(final_df_unique['DCX2_y'].iloc[i])
    elif (dc_list.index(max(dc_list)) == 2) and (final_df_unique['FTX2'].iloc[i] != 1):
        ftx2p.append(0)
    else:
        ftx2p.append(-1)
    
    if (final_df_unique['1.5O_x'].iloc[i] >= 50) and (final_df_unique['1.5O'].iloc[i] == 1):
        over15.append(final_df_unique['1.5O_y'].iloc[i])
    elif (final_df_unique['1.5O_x'].iloc[i] >= 50) and (final_df_unique['1.5O'].iloc[i] != 1):
        over15.append(0)
    else:
        over15.append(-1)
    
    if (final_df_unique['1.5O_x'].iloc[i] < 50) and (final_df_unique['1.5O'].iloc[i] != 1):
        under15.append(final_df_unique['1.5U'].iloc[i])
    elif (final_df_unique['1.5O_x'].iloc[i] < 50) and (final_df_unique['1.5O'].iloc[i] == 1):
        under15.append(0)
    else:
        under15.append(-1)
    
    if (final_df_unique['2.5O_x'].iloc[i] >= 50) and (final_df_unique['2.5O'].iloc[i] == 1):
        over25.append(final_df_unique['2.5O_y'].iloc[i])
    elif (final_df_unique['2.5O_x'].iloc[i] >= 50) and (final_df_unique['2.5O'].iloc[i] != 1):
        over25.append(0)
    else:
        over25.append(-1)
    
    if (final_df_unique['2.5O_x'].iloc[i] < 50) and (final_df_unique['2.5O'].iloc[i] != 1):
        under25.append(final_df_unique['2.5U'].iloc[i])
    elif (final_df_unique['2.5O_x'].iloc[i] < 50) and (final_df_unique['2.5O'].iloc[i] == 1):
        under25.append(0)
    else:
        under25.append(-1)
    
    if (final_df_unique['3.5U_x'].iloc[i] <= 50) and (final_df_unique['3.5U'].iloc[i] != 1):
        over35.append(final_df_unique['3.5O'].iloc[i])
    elif (final_df_unique['3.5U_x'].iloc[i] <= 50) and (final_df_unique['3.5U'].iloc[i] == 1):
        over35.append(0)
    else:
        over35.append(-1)
    
    if (final_df_unique['3.5U_x'].iloc[i] > 50) and (final_df_unique['3.5U'].iloc[i] == 1):
        under35.append(final_df_unique['3.5U_y'].iloc[i])
    elif (final_df_unique['3.5U_x'].iloc[i] > 50) and (final_df_unique['3.5U'].iloc[i] != 1):
        under35.append(0)
    else:
        under35.append(-1)
    
    if (final_df_unique['4.5U_x'].iloc[i] <= 50) and (final_df_unique['4.5U'].iloc[i] != 1):
        over45.append(final_df_unique['4.5O'].iloc[i])
    elif (final_df_unique['4.5U_x'].iloc[i] <= 50) and (final_df_unique['4.5U'].iloc[i] == 1):
        over45.append(0)
    else:
        over45.append(-1)
    
    if (final_df_unique['4.5U_x'].iloc[i] > 50) and (final_df_unique['4.5U'].iloc[i] == 1):
        under45.append(final_df_unique['4.5U_y'].iloc[i])
    elif (final_df_unique['4.5U_x'].iloc[i] > 50) and (final_df_unique['4.5U'].iloc[i] != 1):
        under45.append(0)
    else:
        under45.append(-1)
    
    if (final_df_unique['BTTS_x'].iloc[i] >= 50) and (final_df_unique['BTTS'].iloc[i] == 1):
        btts.append(final_df_unique['BTTS_y'].iloc[i])
    elif (final_df_unique['BTTS_x'].iloc[i] >= 50) and (final_df_unique['BTTS'].iloc[i] != 1):
        btts.append(0)
    else:
        btts.append(-1)
    
    if (final_df_unique['BTTS_x'].iloc[i] < 50) and (final_df_unique['BTTS'].iloc[i] != 1):
        otts.append(final_df_unique['OTTS'].iloc[i])
    elif (final_df_unique['BTTS_x'].iloc[i] < 50) and (final_df_unique['BTTS'].iloc[i] == 1):
        otts.append(0)
    else:
        otts.append(-1)
    
    ht_list = [final_df_unique['HT1_x'].iloc[i], final_df_unique['HTX_x'].iloc[i], final_df_unique['HT2_x'].iloc[i]]
    if (ht_list.index(max(ht_list)) == 0) and (final_df_unique['HT1'].iloc[i] == 1):
        ht1p.append(final_df_unique['HT1_y'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 0) and (final_df_unique['HT1'].iloc[i] != 1):
        ht1p.append(0)
    else:
        ht1p.append(-1)
    
    if (ht_list.index(max(ht_list)) == 1) and (final_df_unique['HTX'].iloc[i] == 1):
        htxp.append(final_df_unique['HTX_y'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 1) and (final_df_unique['HTX'].iloc[i] != 1):
        htxp.append(0)
    else:
        htxp.append(-1)
    
    if (ht_list.index(max(ht_list)) == 2) and (final_df_unique['HT2'].iloc[i] == 1):
        ht2p.append(final_df_unique['HT2_y'].iloc[i])
    elif (ht_list.index(max(ht_list)) == 2) and (final_df_unique['HT2'].iloc[i] != 1):
        ht2p.append(0)
    else:
        ht2p.append(-1)
    
    htdc_list = [final_df_unique['HTDC1X'].iloc[i], final_df_unique['HTDC12'].iloc[i], final_df_unique['HTDCX2'].iloc[i]]
    if (htdc_list.index(max(htdc_list)) == 0) and (final_df_unique['HT1X_y'].iloc[i] == 1):
        ht1xp.append(final_df_unique['HT1X'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 0) and (final_df_unique['HT1X_y'].iloc[i] != 1):
        ht1xp.append(0)
    else:
        ht1xp.append(-1)
    
    if (htdc_list.index(max(htdc_list)) == 1) and (final_df_unique['HT12_y'].iloc[i] == 1):
        ht12p.append(final_df_unique['HT12'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 1) and (final_df_unique['HT12_y'].iloc[i] != 1):
        ht12p.append(0)
    else:
        ht12p.append(-1)
    
    if (htdc_list.index(max(htdc_list)) == 2) and (final_df_unique['HTX2_y'].iloc[i] == 1):
        htx2p.append(final_df_unique['HTX2'].iloc[i])
    elif (htdc_list.index(max(htdc_list)) == 2) and (final_df_unique['HTX2_y'].iloc[i] != 1):
        htx2p.append(0)
    else:
        htx2p.append(-1)
    
    if (final_df_unique['HT0.5O_x'].iloc[i] >= 50) and (final_df_unique['HT0.5O'].iloc[i] == 1):
        htover05.append(final_df_unique['HT0.5O_y'].iloc[i])
    elif (final_df_unique['HT0.5O_x'].iloc[i] >= 50) and (final_df_unique['HT0.5O'].iloc[i] != 1):
        htover05.append(0)
    else:
        htover05.append(-1)
    
    if (final_df_unique['HT0.5O_x'].iloc[i] < 50) and (final_df_unique['HT0.5O'].iloc[i] != 1):
        htunder05.append(final_df_unique['HT0.5U'].iloc[i])
    elif (final_df_unique['HT0.5O_x'].iloc[i] < 50) and (final_df_unique['HT0.5O'].iloc[i] == 1):
        htunder05.append(0)
    else:
        htunder05.append(-1)
    
    if (final_df_unique['HT1.5U_x'].iloc[i] < 50) and (final_df_unique['HT1.5U'].iloc[i] != 1):
        htover15.append(final_df_unique['HT1.5O'].iloc[i])
    elif (final_df_unique['HT1.5U_x'].iloc[i] < 50) and (final_df_unique['HT1.5U'].iloc[i] == 1):
        htover15.append(0)
    else:
        htover15.append(-1)
    
    if (final_df_unique['HT1.5U_x'].iloc[i] >= 50) and (final_df_unique['HT1.5U'].iloc[i] == 1):
        htunder15.append(final_df_unique['HT1.5U_y'].iloc[i])
    elif (final_df_unique['HT1.5U_x'].iloc[i] >= 50) and (final_df_unique['HT1.5U'].iloc[i] != 1):
        htunder15.append(0)
    else:
        htunder15.append(-1)

final_df_unique['FT1P'], final_df_unique['FTXP'], final_df_unique['FT2P'] = ft1p, ftxp, ft2p
final_df_unique['FT1XP'], final_df_unique['FT12P'], final_df_unique['FTX2P'] = ft1xp, ft12p, ftx2p
final_df_unique['1.5OP'], final_df_unique['1.5UP'], final_df_unique['2.5OP'], final_df_unique['2.5UP'] = over15, under15, over25, under25
final_df_unique['3.5OP'], final_df_unique['3.5UP'], final_df_unique['4.5OP'], final_df_unique['4.5UP'] = over35, under35, over45, under45
final_df_unique['BTTSP'], final_df_unique['OTTSP'] = btts, otts
final_df_unique['HT1P'], final_df_unique['HTXP'], final_df_unique['HT2P'] = ht1p, htxp, ht2p
final_df_unique['HT1XP'], final_df_unique['HT12P'], final_df_unique['HTX2P'] = ht1xp, ht12p, htx2p
final_df_unique['HT0.5OP'], final_df_unique['HT0.5UP'] = htover05, htunder05
final_df_unique['HT1.5OP'], final_df_unique['HT1.5UP'] = htover15, htunder15

print('Games Found: ', len(final_df_unique))
final_df_unique.tail()

Games Found:  4474


,League_x,Home,Away,FT1_x,FTX_x,FT2_x,FTR,DC1X_x,DC12_x,DCX2_x,1.5O_x,2.5O_x,3.5U_x,4.5U_x,BTTS_x,HT1_x,HTX_x,HT2_x,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O_x,HT1.5U_x,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U,FT1_y,FTX_y,FT2_y,DC1X_y,DC12_y,DCX2_y,HT1_y,HTX_y,HT2_y,HT1X,HT12,HTX2,BTTS_y,OTTS,1.5O_y,1.5U,2.5O_y,2.5U,3.5O,3.5U_y,4.5O,4.5U_y,HT0.5O_y,HT0.5U,HT1.5O,HT1.5U_y,Date,League_y,FT,HT,FTHG,FTAG,FTTG,HTHG,HTAG,HTTG,FT1,FTX,FT2,FT1X,FT12,FTX2,1.5O,1.5U_y,2.5O,2.5U_y,3.5O_y,3.5U,4.5O_y,4.5U,BTTS,OTTS_y,HT1,HTX,HT2,HT1X_y,HT12_y,HTX2_y,HT0.5O,HT0.5U_y,HT1.5O_y,HT1.5U,FT1P,FTXP,FT2P,FT1XP,FT12P,FTX2P,1.5OP,1.5UP,2.5OP,2.5UP,3.5OP,3.5UP,4.5OP,4.5UP,BTTSP,OTTSP,HT1P,HTXP,HT2P,HT1XP,HT12P,HTX2P,HT0.5OP,HT0.5UP,HT1.5OP,HT1.5UP
4469,Turkey,Konyaspor,Gaziantep,38.80,30.06,31.14,1-1,68.86,69.94,61.20,73.53,46.36,75.14,88.73,52.98,42.77,35.91,21.09,0-0,78.68,63.86,57.00,75.83,59.75,73.95,69.38,38.91,33.15,84.66,88.30,2.05,3.25,3.40,1.30,1.33,1.73,2.75,2.20,4.0,1.22,1.62,1.40,1.73,2.00,1.25,3.75,1.93,1.93,3.25,1.33,6.50,1.11,1.40,2.75,2.75,1.40,2025-03-28 00:00:00,Turkey,1 - 0,(0-0),1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,2.05,-1.0,-1.0,-1.00,1.33,-1.0,0.00,-1.0,-1.00,1.93,-1.0,1.33,-1.0,1.11,0.00,-1.0,0.0,-1.0,-1.0,1.22,-1.0,-1.0,0.00,-1.0,-1.0,1.40
4470,Argentina,Independiente,Godoy Cruz,70.94,22.63,6.38,2-0,93.57,77.32,29.01,79.53,53.10,69.11,84.71,43.38,49.07,38.91,11.72,0-0,87.98,60.79,50.63,71.29,61.97,88.75,45.92,64.23,12.68,62.57,97.49,1.80,3.00,5.50,1.14,1.36,1.95,2.60,1.91,6.0,1.11,1.80,1.50,2.50,1.50,1.57,2.25,2.87,1.37,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Argentina,4 - 0,(4-0),4,0,4,4,0,4,1,0,0,1,1,0,1,0,1,0,1,0,0,1,0,1,1,0,0,1,1,0,1,0,1,0,1.80,-1.0,-1.0,1.14,-1.00,-1.0,1.57,-1.0,2.87,-1.00,-1.0,0.00,-1.0,1.03,-1.00,1.5,2.6,-1.0,-1.0,1.11,-1.0,-1.0,1.62,-1.0,-1.0,0.00
4471,Greece,Panaitolikos,Levadiakos,35.08,34.61,30.32,0-0,69.69,65.40,64.93,47.55,21.92,91.98,97.58,30.22,20.00,59.27,20.73,0-0,79.27,40.73,80.00,48.38,83.24,57.11,53.38,20.80,17.81,94.56,95.77,2.20,3.10,3.50,1.29,1.36,1.67,3.10,1.83,4.5,1.17,1.83,1.36,2.25,1.57,1.57,2.25,2.88,1.40,6.00,1.13,15.00,1.03,1.62,2.20,4.00,1.22,2025-03-29 00:00:00,Greece,1 - 2,(1-0),1,2,3,1,0,1,0,0,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,1,0,1,0,0,1,0.00,-1.0,-1.0,0.00,-1.00,-1.0,-1.00,0.0,-1.00,0.00,-1.0,1.13,-1.0,1.03,-1.00,0.0,-1.0,0.0,-1.0,-1.00,-1.0,0.0,-1.00,0.0,-1.0,1.22
4472,Switzerland,Sion,Yverdon,54.91,24.30,20.77,1-1,79.21,75.68,45.07,76.33,51.72,70.42,85.62,52.38,47.86,38.43,13.43,0-0,86.29,61.29,51.86,71.87,62.29,82.69,62.84,52.34,26.06,74.28,92.14,1.48,4.50,6.25,1.13,1.18,2.63,2.00,2.40,6.0,1.11,1.53,1.73,1.83,1.83,1.22,4.00,1.73,2.08,2.75,1.40,5.50,1.14,1.33,3.25,2.50,1.50,2025-03-29 00:00:00,Switzerland,1 - 1,(0-1),1,1,2,0,1,1,0,1,0,1,0,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1,1,1,0,0,1,0.00,-1.0,-1.0,1.13,-1.00,-1.0,1.22,-1.0,0.00,-1.00,-1.0,1.40,-1.0,1.14,1.83,-1.0,0.0,-1.0,-1.0,0.00,-1.0,-1.0,1.33,-1.0,-1.0,1.50
4473,Norway,Bodo/Glimt,Viking,72.20,13.90,13.42,3-1,86.10,85.62,27.32,93.02,81.62,35.05,54.13,70.08,73.46,25.19,0.00,1-0,98.65,73.46,25.19,73.46,59.93,95.06,73.52,81.19,38.62,39.91,84.32,1.45,5.00,5.75,1.15,1.18,2.63,1.83,2.80,5.0,1.14,1.40,1.83,1.44,2.63,1.08,8.00,1.33,3.40,1.80,1.91,2.75,1.40,1.18,4.50,1.90,1.90,2025-04-06 00:00:00,Norway,1 - 0,(0-0),1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,1.45,-1.0,-1.0,1.15,-1.00,-1.0,0.00,-1.0,0.00,-1.00,0.0,-1.00,-1.0,1.40,0.00,-1.0,0.0,-1.0,-1.0,1.14,-1.0,-1.0,0.00,-1.0,-1.0,1.90


# Checking For ROI of Profit Columns

In [7]:
# Select columns that end with 'P'
columns = [col for col in final_df_unique.columns if col.endswith('P')]

# Initialize lists to store results and games
results, games_list = [],  []

for col in columns:
    my_df = final_df_unique[final_df_unique[col] >= 0]
    numerator = np.sum(my_df[col]) - len(my_df)
    result = round(numerator / len(my_df) * 100, 2)
    
    # Append results and games to respective lists
    results.append(result)
    games_list.append(len(my_df))

# Convert results to a DataFrame with an additional column for Games
results_df = pd.DataFrame({
    'Column': columns,
    'ROI': results,
    'Games': games_list
})
results_df

,Column,ROI,Games
0,FT1P,-5.24,2563
1,FTXP,-8.44,291
2,FT2P,-11.48,1620
3,FT1XP,-4.11,1971
4,FT12P,-5.75,1324
5,FTX2P,-7.63,1179
6,1.5OP,-5.81,4131
7,1.5UP,-4.03,343
8,2.5OP,-4.58,2101
9,2.5UP,-4.23,2373


# ROI of Profit Columns According To Leagues

In [8]:
# Step 1: Filter leagues with at least 10 games
league_counts = final_df_unique['League_x'].value_counts()
leagues_with_10_games = league_counts[league_counts >= 10].index
filtered_df = final_df_unique[final_df_unique['League_x'].isin(leagues_with_10_games)]

# Group by 'League' and calculate results for each group
grouped_results = {}
for league, group in filtered_df.groupby('League_x'):
    group_results = {}
    for col in columns:
        my_df = group[group[col] >= 0]
        numerator = np.sum(my_df[col]) - len(my_df)
        group_results[col] = round(numerator / len(my_df) * 100, 2)
    # Add the number of games for this league
    group_results['Games'] = round(len(group),2)
    grouped_results[league] = group_results

# Convert grouped results to a DataFrame for better visualization
grouped_results_df = pd.DataFrame(grouped_results).T

# Define a function to apply conditional formatting
def highlight_positive(val):
    # Highlight background to red if the value is positive
    color = 'background-color: red' if isinstance(val, (int, float)) and val > 0 else ''
    return color

# Apply the function to the DataFrame
styled_df = (
    grouped_results_df.style
    .applymap(highlight_positive)
    .format("{:.2f}")  # Format only numeric columns, excluding 'Games'
)

# Save the styled DataFrame to Excel
styled_df.to_excel("ROI_leagues.xlsx", index=True)

# Display the styled DataFrame
styled_df

,FT1P,FTXP,FT2P,FT1XP,FT12P,FTX2P,1.5OP,1.5UP,2.5OP,2.5UP,3.5OP,3.5UP,4.5OP,4.5UP,BTTSP,OTTSP,HT1P,HTXP,HT2P,HT1XP,HT12P,HTX2P,HT0.5OP,HT0.5UP,HT1.5OP,HT1.5UP,Games
Argentina,-33.08,8.67,-29.11,-4.36,-66.00,-4.65,0.16,-11.13,-2.08,-5.17,-100.00,-4.60,nan,-2.43,-2.22,-1.12,-46.82,14.33,78.83,-5.31,-100.00,7.75,9.92,12.57,-100.00,-15.88,60.00
Australia,-44.59,nan,-18.77,2.78,-12.07,10.71,-3.17,nan,6.52,51.22,-34.33,-34.86,nan,-12.17,9.26,-24.55,0.30,-1.58,-11.50,2.47,nan,-20.73,-7.23,nan,20.43,-25.70,30.00
Austria,-30.36,-22.50,-24.82,-5.07,9.75,8.89,2.68,300.00,26.09,-4.17,5.91,4.61,-100.00,-2.43,-13.34,-36.76,-50.09,-9.47,-33.13,-5.26,nan,-3.96,4.69,62.78,6.92,1.22,57.00
Belgium,9.54,-2.14,-32.44,5.43,-36.12,-9.12,-9.19,-35.00,-9.31,5.93,-31.42,7.88,-100.00,0.89,-17.76,-1.10,-10.98,-13.48,-19.46,1.67,-68.60,-9.46,-7.32,23.29,-77.37,0.37,118.00
Brazil,-15.08,nan,-54.25,18.67,-66.00,-70.00,-3.53,nan,110.00,-10.93,nan,-3.29,nan,-3.94,23.00,-4.50,-8.50,-45.67,nan,-10.75,nan,-44.80,10.71,nan,nan,-7.29,17.00
Colombia,22.64,-22.50,-57.14,-9.21,-35.50,14.33,-29.11,51.14,-37.43,27.83,-100.00,11.81,-100.00,1.13,-58.00,18.45,16.75,-17.43,-100.00,-7.47,nan,27.25,-7.83,-64.29,-7.50,-9.65,25.00
Denmark,-13.15,-100.00,-18.08,-6.64,-25.25,-10.95,0.94,nan,13.11,40.25,-32.70,-21.60,-100.00,-6.27,19.17,-8.06,13.62,4.17,8.50,7.13,-22.00,-0.37,-6.84,-100.00,42.88,1.42,53.00
England,-5.75,-26.67,-16.66,-7.37,8.91,-0.98,-3.16,-100.00,-9.77,-8.94,-54.07,-6.94,-100.00,2.92,-5.73,-14.27,-12.46,-0.67,27.51,-6.99,22.83,-18.65,3.02,-2.78,1.65,1.42,133.00
England2,-7.74,13.46,-19.16,1.51,-1.93,-0.11,-4.57,6.29,-9.19,5.98,-24.33,1.88,nan,0.93,-6.74,-9.00,7.77,-9.88,-71.56,0.08,nan,-15.23,-5.77,3.96,31.65,-7.24,269.00
England3,2.01,49.85,-6.57,-2.86,-3.86,-11.25,-8.75,0.62,-3.50,6.03,-30.32,2.43,-46.43,-1.99,-1.71,3.28,-31.97,-11.97,-3.07,-3.88,15.00,-13.01,-8.58,-17.21,-2.63,-2.16,281.00


# Creating Optimum Threshold for Each Prediction Column

In [9]:
# Assuming `df` is your DataFrame and it contains the columns for percentages and correctness
def calculate_threshold(percentages, predictions):
    # Ensure inputs are pandas Series
    percentages = pd.Series(percentages)
    predictions = pd.Series(predictions)
    
    thresholds = percentages.unique()
    best_threshold = 0
    best_j_stat = -np.inf  # Start with negative infinity for comparison
    
    for threshold in thresholds:
        # Predict 1s based on the threshold
        predicted_1s = (percentages >= threshold).astype(int)
        
        # Calculate true positives, true negatives, false positives, false negatives
        true_positives = ((predicted_1s == 1) & (predictions == 1)).sum()
        true_negatives = ((predicted_1s == 0) & (predictions == 0)).sum()
        false_positives = ((predicted_1s == 1) & (predictions == 0)).sum()
        false_negatives = ((predicted_1s == 0) & (predictions == 1)).sum()
        
        # Calculate Sensitivity (Recall) and Specificity
        sensitivity = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
        specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0
        
        # Calculate Youden's J Statistic
        j_stat = sensitivity + specificity - 1
        
        # Update best threshold if J statistic improves
        if j_stat > best_j_stat:
            best_j_stat = j_stat
            best_threshold = threshold
    
    return best_threshold, round(best_j_stat, 2)

# Select only numeric columns
numeric_columns = final_df_unique.select_dtypes(include=[np.number])

# Remove rows where any numeric value is greater than 100
final_df_unique = final_df_unique[(numeric_columns <= 100).all(axis=1)]

#Selecting dataframes with model predictions
ft1df = final_df_unique[final_df_unique['FT1_x'] >= final_df_unique[['FTX_x', 'FT2_x']].max(axis=1)]
ftxdf = final_df_unique[final_df_unique['FTX_x'] >= final_df_unique[['FT1_x', 'FT2_x']].max(axis=1)]
ft2df = final_df_unique[final_df_unique['FT2_x'] >= final_df_unique[['FTX_x', 'FT1_x']].max(axis=1)]
dc1xdf = final_df_unique[final_df_unique['DC1X_x'] >= final_df_unique[['DC12_x', 'DCX2_x']].max(axis=1)]
dc12df = final_df_unique[final_df_unique['DC12_x'] >= final_df_unique[['DC1X_x', 'DCX2_x']].max(axis=1)]
dcx2df = final_df_unique[final_df_unique['DCX2_x'] >= final_df_unique[['DC1X_x', 'DC12_x']].max(axis=1)]
over15df, under15df = final_df_unique[final_df_unique['1.5O_x'] >= 50], final_df_unique[final_df_unique['1.5O_x'] < 50]
over25df, under25df = final_df_unique[final_df_unique['2.5O_x'] >= 50], final_df_unique[final_df_unique['2.5O_x'] < 50]
over35df, under35df = final_df_unique[final_df_unique['3.5U_x'] < 50], final_df_unique[final_df_unique['3.5U_x'] >= 50]
over45df, under45df = final_df_unique[final_df_unique['4.5U_x'] < 50], final_df_unique[final_df_unique['4.5U_x'] >= 50]
bttsdf, ottsdf = final_df_unique[final_df_unique['BTTS_x'] >= 50], final_df_unique[final_df_unique['BTTS_x'] < 50]
ht1df = final_df_unique[final_df_unique['HT1_x'] >= final_df_unique[['HTX_x', 'HT2_x']].max(axis=1)]
htxdf = final_df_unique[final_df_unique['HTX_x'] >= final_df_unique[['HT1_x', 'HT2_x']].max(axis=1)]
ht2df = final_df_unique[final_df_unique['HT2_x'] >= final_df_unique[['HT1_x', 'HTX_x']].max(axis=1)]
ht1xdf = final_df_unique[final_df_unique['HTDC1X'] >= final_df_unique[['HTDC12', 'HTDCX2']].max(axis=1)]
ht12df = final_df_unique[final_df_unique['HTDC12'] >= final_df_unique[['HTDC1X', 'HTDCX2']].max(axis=1)]
htx2df = final_df_unique[final_df_unique['HTDCX2'] >= final_df_unique[['HTDC1X', 'HTDC12']].max(axis=1)]
htover05df, htunder05df = final_df_unique[final_df_unique['HT0.5O_x'] >= 50], final_df_unique[final_df_unique['HT0.5O_x'] < 50]
htover15df, htunder15df = final_df_unique[final_df_unique['HT1.5U_x'] < 50], final_df_unique[final_df_unique['HT1.5U_x'] >= 50]

ft1t, ft1a = calculate_threshold(ft1df['FT1_x'], ft1df['FT1'])
ftxt, ftxa = calculate_threshold(ftxdf['FTX_x'], ftxdf['FTX'])
ft2t, ft2a = calculate_threshold(ft2df['FT2_x'], ft2df['FT2'])
ft1xt, ft1xa = calculate_threshold(dc1xdf['DC1X_x'], dc1xdf['FT1X'])
ft12t, ft12a = calculate_threshold(dc12df['DC12_x'], dc12df['FT12'])
ftx2t, ftx2a = calculate_threshold(dcx2df['DCX2_x'], dcx2df['FTX2'])
over15t, over15a = calculate_threshold(over15df['1.5O_x'], over15df['1.5O'])
under15t, under15a = calculate_threshold(under15df['1.5O_x'], under15df['1.5U'])
over25t, over25a = calculate_threshold(over25df['2.5O_x'], over25df['2.5O'])
under25t, under25a = calculate_threshold(under25df['2.5O_x'], under25df['2.5U'])
over35t, over35a = calculate_threshold(over35df['3.5U_x'], over35df['3.5O'])
under35t, under35a = calculate_threshold(under35df['3.5U_x'], under35df['3.5U'])
over45t, over45a = calculate_threshold(over45df['4.5U_x'], over45df['4.5O'])
under45t, under45a = calculate_threshold(under45df['4.5U_x'], under45df['4.5U'])
bttst, bttsa = calculate_threshold(bttsdf['BTTS_x'], bttsdf['BTTS'])
ottst, ottsa = calculate_threshold(ottsdf['BTTS_x'], ottsdf['OTTS'])
ht1t, ht1a = calculate_threshold(ht1df['HT1_x'], ht1df['HT1'])
htxt, htxa = calculate_threshold(htxdf['HTX_x'], htxdf['HTX'])
ht2t, ht2a = calculate_threshold(ht2df['HT2_x'], ht2df['HT2'])
ht1xt, ht1xa = calculate_threshold(ht1xdf['HTDC1X'], ht1xdf['HT1X'])
ht12t, ht12a = calculate_threshold(ht12df['HTDC12'], ht12df['HT12'])
htx2t, htx2a = calculate_threshold(htx2df['HTDCX2'], htx2df['HTX2'])
htover05t, htover05a = calculate_threshold(htover05df['HT0.5O_x'], htover05df['HT0.5O'])
htunder05t, htunder05a = calculate_threshold(htunder05df['HT0.5O_x'], htunder05df['HT0.5U'])
htover15t, htover15a = calculate_threshold(htover15df['HT1.5U_x'], htover15df['HT1.5O'])
htunder15t, htunder15a = calculate_threshold(htunder15df['HT1.5U_x'], htunder15df['HT1.5U'])

new_ft1df, new_ftxdf, new_ft2df = ft1df[ft1df['FT1_x'] >= ft1t],ftxdf[ftxdf['FTX_x'] >= ftxt],ft2df[ft2df['FT2_x'] >= ft2t]
new_ft1xdf, new_ft12df, new_ftx2df = dc1xdf[dc1xdf['DC1X_x'] >= ft1xt],dc12df[dc12df['DC12_x'] >= ft12t],dcx2df[dcx2df['DCX2_x'] >= ftx2t] 
new_over15, new_under15 = over15df[over15df['1.5O_x'] >= over15t], under15df[under15df['1.5O_x'] <= under15t]
new_over25, new_under25 = over25df[over25df['2.5O_x'] >= over25t], under25df[under25df['2.5O_x'] <= under25t]
new_over35, new_under35 = over35df[over35df['3.5U_x'] <= over35t], under35df[under35df['3.5U_x'] >= under35t]
new_over45, new_under45 = over45df[over45df['4.5U_x'] <= over45t], under45df[under45df['4.5U_x'] >= under45t]
new_btts, new_otts = bttsdf[bttsdf['BTTS_x'] >= bttst], ottsdf[ottsdf['BTTS_x'] <= ottst]
new_ht1df, new_htxdf, new_ht2df = ht1df[ht1df['HT1_x'] >= ht1t],htxdf[htxdf['HTX_x'] >= htxt],ht2df[ht2df['HT2_x'] >= ht2t]
new_ht1xdf, new_ht12df, new_htx2df = ht1xdf[ht1xdf['HTDC1X'] >= ht1xt],ht12df[ht12df['HTDC12'] >= ht12t],htx2df[htx2df['HTDCX2'] >= htx2t]
new_htover05, new_htunder05 = htover05df[htover05df['HT0.5O_x'] >= htover05t], htunder05df[htunder05df['HT0.5O_x'] <= htunder05t]
new_htover15, new_htunder15 = htover15df[htover15df['HT1.5U_x'] <= htover15t], htunder15df[htunder15df['HT1.5U_x'] >= htunder15t]

# Store the results in a list
results = [
('FT1', ft1t, ft1a, len(new_ft1df), round(len(new_ft1df)/len(ft1df)*100,2), np.sum(new_ft1df['FT1P']) - len(new_ft1df)),
('FTX', ftxt, ftxa, len(new_ftxdf), round(len(new_ftxdf)/len(ftxdf)*100,2), np.sum(new_ftxdf['FTXP']) - len(new_ftxdf)),
('FT2', ft2t, ft2a, len(new_ft2df), round(len(new_ft2df)/len(ft2df)*100,2), np.sum(new_ft2df['FT2P']) - len(new_ft2df)),
('FT1X', ft1xt, ft1xa, len(new_ft1xdf), round(len(new_ft1xdf)/len(dc1xdf)*100,2), np.sum(new_ft1xdf['FT1XP']) - len(new_ft1xdf)),
('FT12', ft12t, ft12a, len(new_ft12df), round(len(new_ft12df)/len(dc12df)*100,2), np.sum(new_ft12df['FT12P']) - len(new_ft12df)),
('FTX2', ftx2t, ftx2a, len(new_ftx2df), round(len(new_ftx2df)/len(dcx2df)*100,2), np.sum(new_ftx2df['FTX2P']) - len(new_ftx2df)),
('1.5O', over15t, over15a, len(new_over15), round(len(new_over15)/len(over15df)*100,2), np.sum(new_over15['1.5OP'])-len(new_over15)),
('1.5U', under15t, under15a, len(new_under15), round(len(new_under15)/len(under15df)*100,2), np.sum(new_under15['1.5UP'])-len(new_under15)),
('2.5O', over25t, over25a, len(new_over25), round(len(new_over25)/len(over25df)*100,2), np.sum(new_over25['2.5OP'])-len(new_over25)),
('2.5U', under25t, under25a, len(new_under25), round(len(new_under25)/len(under25df)*100,2), np.sum(new_under25['2.5UP'])-len(new_under25)),
('3.5O', over35t, over35a, len(new_over35), round(len(new_over35)/len(over35df)*100,2), np.sum(new_over35['3.5OP'])-len(new_over35)),
('3.5U', under35t, under35a, len(new_under35), round(len(new_under35)/len(under35df)*100,2), np.sum(new_under35['3.5UP'])-len(new_under35)),
('4.5O', over45t, over45a, len(new_over45), round(len(new_over45)/len(over45df)*100,2), np.sum(new_over45['4.5OP'])-len(new_over45)),
('4.5U', under45t, under45a, len(new_under45), round(len(new_under45)/len(under45df)*100,2), np.sum(new_under45['4.5UP'])-len(new_under45)),
('BTTS', bttst, bttsa, len(new_btts), round(len(new_btts)/len(bttsdf)*100,2), np.sum(new_btts['BTTSP'])-len(new_btts)),
('OTTS', ottst, ottsa, len(new_otts), round(len(new_otts)/len(ottsdf)*100,2), np.sum(new_otts['OTTSP'])-len(new_otts)),
('HT1', ht1t, ht1a, len(new_ht1df), round(len(new_ht1df)/len(ht1df)*100,2), np.sum(new_ht1df['HT1P']) - len(new_ht1df)),
('HTX', htxt, htxa, len(new_htxdf), round(len(new_htxdf)/len(htxdf)*100,2), np.sum(new_htxdf['HTXP']) - len(new_htxdf)),
('HT2', ht2t, ht2a, len(new_ht2df), round(len(new_ht2df)/len(ht2df)*100,2), np.sum(new_ht2df['HT2P']) - len(new_ht2df)),
('HT1X', ht1xt, ht1xa, len(new_ht1xdf), round(len(new_ht1xdf)/len(ht1xdf)*100,2), np.sum(new_ht1xdf['HT1XP']) - len(new_ht1xdf)),
('HT12', ht12t, ht12a, len(new_ht12df), round(len(new_ht12df)/len(ht12df)*100,2), np.sum(new_ht12df['HT12P']) - len(new_ht12df)),
('HTX2', htx2t, htx2a, len(new_htx2df), round(len(new_htx2df)/len(htx2df)*100,2), np.sum(new_htx2df['HTX2P']) - len(new_htx2df)),
('HT0.5O', htover05t, htover05a, len(new_htover05), round(len(new_htover05)/len(htover05df)*100,2), np.sum(new_htover05['HT0.5OP'])-len(new_htover05)),
('HT0.5U', htunder05t, htunder05a, len(new_htunder05), round(len(new_htunder05)/len(htunder05df)*100,2), np.sum(new_htunder05['HT0.5UP'])-len(new_htunder05)),
('HT1.5O', htover15t, htover15a, len(new_htover15), round(len(new_htover15)/len(htover15df)*100,2), np.sum(new_htover15['HT1.5OP'])-len(new_htover15)),
('HT1.5U', htunder15t, htunder15a, len(new_htunder15), round(len(new_htunder15)/len(htunder15df)*100,2), np.sum(new_htunder15['HT1.5UP'])-len(new_htunder15))
]

# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['Prediction', 'Threshold', 'J-Stat', 'Games', 'Games%', 'Profit'])
results_df['ROI'] = round(results_df['Profit'] / results_df['Games'] * 100, 2)
print('Number of matches: ', len(final_df_unique))
results_df

Number of matches:  4434


,Prediction,Threshold,J-Stat,Games,Games%,Profit,ROI
0,FT1,58.80,0.18,1021,40.12,14.77,1.45
1,FTX,53.39,0.06,31,11.23,9.30,30.00
2,FT2,48.72,0.11,823,50.96,-73.36,-8.91
3,FT1X,87.80,0.13,653,33.59,-15.84,-2.43
4,FT12,75.59,0.08,737,55.71,-19.62,-2.66
5,FTX2,80.19,0.12,607,52.01,-35.22,-5.80
6,1.5O,76.29,0.07,1933,47.22,-106.65,-5.52
7,1.5U,30.21,-1.00,53,15.59,4.82,9.09
8,2.5O,64.74,0.06,849,40.72,-29.12,-3.43
9,2.5U,7.67,-1.00,38,1.62,-2.30,-6.05


# Testing Best / Most Profitable Model Predictions

In [10]:
# Select columns from predictions table
predictions = ['FT1_x', 'FTX_x', 'FT2_x', 'DC1X_x', 'DC12_x', 'DCX2_x', 
               '1.5O_x', '2.5O_x', '3.5U_x', '4.5U_x', 'BTTS_x', 
               'HT1_x', 'HTX_x', 'HT2_x', 'HTDC1X', 'HTDC12', 'HTDCX2', 
               'HT0.5O_x', 'HT1.5U_x']

# Select columns from betting odds table
results = ['FT1', 'FTX', 'FT2', 'FT1X', 'FT12', 'FTX2', 
           '1.5O', '2.5O', '3.5U', '4.5U', 'BTTS',
            'HT1', 'HTX', 'HT2', 'HT1X', 'HT12', 'HTX2', 
            'HT0.5O', 'df2_HT1.5U']

# Select columns ending with 'P' (profit columns)
profits = ['FT1P', 'FTXP', 'FT2P', 'FT1XP', 'FT12P', 'FTX2P', 
           '1.5OP', '2.5OP', '3.5UP', '4.5UP', 'BTTSP',
            'HT1P', 'HTXP', 'HT2P', 'HT1XP', 'HT12P', 'HTX2P', 
            'HT0.5OP', 'HT1.5UP']

bet, percentage, profit = [], [], []

for i in range(len(final_df_unique)):
    my_list = []
    for j in predictions:
        my_list.append(final_df_unique[j].iloc[i])
    percentage.append(max(my_list))
    max_index = my_list.index(max(my_list))
    bet.append(results[max_index])
    profit_column = profits[max_index]
    profit.append(final_df_unique[profit_column].iloc[i])

# Create a DataFrame
model_recs = pd.DataFrame({
    'League': final_df_unique['League_x'],
    'Home': final_df_unique['Home'],
    'Away': final_df_unique['Away'],
    'BET': bet,
    'Percentage': percentage,
    'Profit': profit
})


print('Matches found: ', len(final_df_unique))
print(f"Correct Predictions: {len(model_recs[model_recs['Profit'] > 0])/len(model_recs)*100}")
print(f"Profit: {round(sum(model_recs['Profit']) - len(model_recs),2)} ROI: {round((sum(model_recs['Profit']) - len(model_recs)) / len(model_recs) * 100, 2)}%")
model_recs.tail()

Matches found:  4434
Correct Predictions: 84.07758231844835
Profit: -170.09 ROI: -3.84%


,League,Home,Away,BET,Percentage,Profit
4469,Turkey,Konyaspor,Gaziantep,4.5U,88.73,1.11
4470,Argentina,Independiente,Godoy Cruz,FT1X,93.57,1.14
4471,Greece,Panaitolikos,Levadiakos,4.5U,97.58,1.03
4472,Switzerland,Sion,Yverdon,HT1X,86.29,0.00
4473,Norway,Bodo/Glimt,Viking,HT1X,98.65,1.14


In [11]:
final_df_unique['OTTS_x'] = 100 - final_df_unique['BTTS_x']
final_df_unique['1.5U_x'] = 100 - final_df_unique['1.5O_x']
final_df_unique['2.5U_x'] = 100 - final_df_unique['2.5O_x']
final_df_unique['3.5O_x'] = 100 - final_df_unique['3.5U_x']
final_df_unique['4.5O_x'] = 100 - final_df_unique['4.5U_x']
final_df_unique['HT0.5U_x'] = 100 - final_df_unique['HT0.5O_x']
final_df_unique['HT1.5O_x'] = 100 - final_df_unique['HT1.5U_x'] 

# Select columns starting with 'df1_'
predictions = ['FT1_x', 'FTX_x', 'FT2_x', 'DC1X_x', 'DC12_x', 'DCX2_x', 
               '1.5O_x', '1.5U_x', '2.5O_x','2.5U_x','3.5O_x', '3.5U_x', 
               '4.5O_x', '4.5U_x', 'BTTS_x', 'OTTS_x',
               'HT1_x', 'HTX_x', 'HT2_x', 'HTDC1X', 'HTDC12', 'HTDCX2', 
               'HT0.5O_x', 'HT0.5U_x', 'HT1.5O_x', 'HT1.5U_x']

# Select columns starting with 'df2_'
odds = ['FT1_y', 'FTX_y', 'FT2_y', 'DC1X_y', 'DC12_y', 'DCX2_y', 
           '1.5O_y', '1.5U', '2.5O_y','2.5U', '3.5O','3.5U_y', 
           '4.5O', '4.5U_y', 'BTTS_y', 'OTTS',
            'HT1_y', 'HTX_y', 'HT2_y', 'HT1X', 'HT12', 'HTX2', 
            'HT0.5O_y', 'HT0.5U', 'HT1.5O', 'HT1.5U_y']

# Select columns ending with 'P'
profit = ['FT1P', 'FTXP', 'FT2P', 'FT1XP', 'FT12P', 'FTX2P', 
           '1.5OP', '1.5UP', '2.5OP', '2.5UP', '3.5OP', '3.5UP', '4.5OP', '4.5UP', 
           'BTTSP', 'OTTSP', 'HT1P', 'HTXP', 'HT2P', 'HT1XP', 'HT12P', 'HTX2P', 
            'HT0.5OP', 'HT0.5UP', 'HT1.5OP','HT1.5UP']

bets, percentages, profits, difference = [], [], [], [] 
for i in range(len(final_df_unique)):
    my_list = []
    valid_indices = []  # To keep track of indices where profit is not negative
    for j in range(len(predictions)):
        pred_column = predictions[j]
        odds_column = odds[j]
        profit_column = profit[j]  # Corresponding profit column
        
        # Calculate the value
        my_value = (100 / final_df_unique[pred_column].iloc[i]) - final_df_unique[odds_column].iloc[i]
        
        # Only add to the list if the corresponding profit is non-negative
        if final_df_unique[profit_column].iloc[i] >= 0:
            my_list.append(my_value)
            valid_indices.append(j)
        else:
            my_list.append(float('-inf'))  # Set to negative infinity to ignore in max()
    
    if valid_indices:  # Ensure there is at least one valid index
        max_index = my_list.index(max(my_list))
        rec_bet = profit[max_index]
        percent_bet = predictions[max_index]
        bets.append(rec_bet)
        percentages.append(final_df_unique[percent_bet].iloc[i])
        rec_profit = profit[max_index]
        profits.append(final_df_unique[rec_profit].iloc[i])
        difference.append(round(max(my_list), 2))
    else:
        # Handle case where no valid profits are found for this match
        bets.append(None)
        percentages.append(None)
        profits.append(None)
        difference.append(None)

# Create a DataFrame
model_recs = pd.DataFrame({
    'League': final_df_unique['League_x'],
    'Home': final_df_unique['Home'],
    'Away': final_df_unique['Away'],
    'BET': bets,
    'Percentage': percentages,
    'Profit': profits,
    'Difference': difference
}).dropna()  # Drop rows with None values

print('Matches found: ', len(final_df_unique))
print(f"Correct Predictions: {len(model_recs[model_recs['Profit'] > 0])/len(model_recs)*100}")
print(f"Profit: {round(sum(model_recs['Profit']) - len(model_recs), 2)} ROI: {round((sum(model_recs['Profit']) - len(model_recs)) / len(model_recs) * 100, 2)}%")
model_recs.tail()

Matches found:  4434
Correct Predictions: 67.20793865584123
Profit: -181.86 ROI: -4.1%


,League,Home,Away,BET,Percentage,Profit,Difference
4469,Turkey,Konyaspor,Gaziantep,FT1P,38.80,2.05,0.53
4470,Argentina,Independiente,Godoy Cruz,HT1.5UP,61.97,0.00,0.39
4471,Greece,Panaitolikos,Levadiakos,FT1P,35.08,0.00,0.65
4472,Switzerland,Sion,Yverdon,FT1P,54.91,0.00,0.34
4473,Norway,Bodo/Glimt,Viking,4.5UP,54.13,1.40,0.45


## Checking the Betting Strategy in Article 

probability(model) / probability(bookies) > r (between 1 and 1.5)

In [12]:
def find_best_threshold_for_column(df, r_col, p_col, start=1.0, stop=1.5, step=0.01):
    """
    For a given r column and probability column in df, loop through thresholds
    from start to stop (inclusive) in step increments. Compute the score for each threshold,
    and return the threshold that gives the maximum score along with that score.
    
    Score is defined as:
      (sum(probabilities) - count(probabilities)) / count(probabilities) * 100
    """
    best_score = -np.inf
    best_threshold = None
    # Create thresholds from start to stop inclusive
    for thr in np.arange(start, stop + step, step):
        # Filter the rows based on the current threshold and probability condition
        subset = df[(df[r_col] > thr) & (df[p_col] >= 0)]
        if len(subset) == 0:
            continue  # skip if no data for this threshold
        # Convert probability values from percentage to decimals by dividing by 100.
        score = ((subset[p_col]).sum() - len(subset)) / len(subset) * 100
        if score > best_score:
            best_score = score
            best_threshold = thr
            games = len(subset)
    return best_threshold, best_score, games

# Step 1: Compute r score columns (if not already computed)
for pred, odd in zip(predictions, odds):
    r_col = f'{pred}_r'
    final_df_unique[r_col] = (final_df_unique[pred] / 100) / (1 / final_df_unique[odd])

# Dictionary to store the best threshold and score for each column
results = {}

for prof, pred in zip(profit, predictions):
    r_col = f'{pred}_r'
    # profit column is taken directly from the profit list.
    profit_col = prof
    
    # Optional: check if columns exist
    if r_col not in final_df_unique.columns:
        print(f"Column {r_col} not found, skipping.")
        continue
    if profit_col not in final_df_unique.columns:
        print(f"Column {profit_col} not found, skipping.")
        continue

    best_thr, best_score, games = find_best_threshold_for_column(final_df_unique, r_col, profit_col)
    results[prof] = {'best_threshold': best_thr, 'best_score': best_score, 'games': games}

# Convert the results dictionary to a DataFrame for viewing.
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by="best_score", ascending=False)
results_df

,best_threshold,best_score,games
HT12P,1.44,45.333333,6.0
4.5UP,1.31,36.000000,1.0
HT0.5OP,1.37,31.846154,13.0
HT1XP,1.50,26.500000,4.0
3.5UP,1.48,19.923077,13.0
1.5OP,1.33,11.625000,8.0
BTTSP,1.35,8.452381,42.0
2.5OP,1.49,7.380000,100.0
OTTSP,1.45,6.282828,198.0
HTX2P,1.38,6.094340,53.0


# Combining Different Bets

In [13]:
# Define the list of base columns and multiplier columns
ft_result_columns = ['FT1P', 'FTXP', 'FT2P', 'FT1XP', 'FT12P', 'FTX2P'] 
ft_goals_columns = ['1.5OP', '1.5UP', '2.5OP', '2.5UP', '3.5OP', '3.5UP', '4.5OP', '4.5UP', 'BTTSP', 'OTTSP']
ht_result_columns = ['HT1P', 'HTXP', 'HT2P', 'HT1XP', 'HT12P', 'HTX2P']
ht_goals_columns = ['HT0.5OP', 'HT0.5UP', 'HT1.5OP','HT1.5UP']

# List to store summary information for each new column.
summary_list = []

# Multiply each FT result column with each FT goals column.
for ft_result in ft_result_columns:
    for ft_goal in ft_goals_columns:
        new_col = f"{ft_result}_{ft_goal}"  # e.g., "FT1P_1.5OP"
        # Multiply only if both factors are >= 0; otherwise assign -1.
        final_df_unique[new_col] = np.where(
            (final_df_unique[ft_result] >= 0) & (final_df_unique[ft_goal] >= 0),
            final_df_unique[ft_result] * final_df_unique[ft_goal],
            -1
        )
        # Select only valid (>= 0) product values.
        valid_values = final_df_unique.loc[final_df_unique[new_col] >= 0, new_col]
        count_valid = len(valid_values)
        summary_value = (valid_values.sum() - count_valid) / count_valid if count_valid > 0 else np.nan
        summary_list.append({'Combination': new_col, 'ROI': summary_value, 'Games': count_valid})
        # Add a new column to store the count of valid games.
        final_df_unique[f"{new_col}_games"] = count_valid

# Multiply each HT result column with each HT goals column.
for ht_result in ht_result_columns:
    for ht_goal in ht_goals_columns:
        new_col = f"{ht_result}_{ht_goal}"  # e.g., "HT1P_HT0.5OP"
        final_df_unique[new_col] = np.where(
            (final_df_unique[ht_result] >= 0) & (final_df_unique[ht_goal] >= 0),
            final_df_unique[ht_result] * final_df_unique[ht_goal],
            -1
        )
        valid_values = final_df_unique.loc[final_df_unique[new_col] >= 0, new_col]
        count_valid = len(valid_values)
        summary_value = (valid_values.sum() - count_valid) / count_valid if count_valid > 0 else np.nan
        summary_list.append({'Combination': new_col, 'ROI': summary_value, 'Games': count_valid})
        final_df_unique[f"{new_col}_games"] = count_valid

# Extra combinations: BTTSP with 2.5OP and BTTSP with 4.5UP.
extra_combinations = [("BTTSP_2.5OP", "2.5OP"), ("BTTSP_4.5UP", "4.5UP")]

for new_col, other_col in extra_combinations:
    final_df_unique[new_col] = np.where(
        (final_df_unique["BTTSP"] >= 0) & (final_df_unique[other_col] >= 0),
        final_df_unique["BTTSP"] * final_df_unique[other_col],
        -1
    )
    valid_values = final_df_unique.loc[final_df_unique[new_col] >= 0, new_col]
    count_valid = len(valid_values)
    summary_value = (valid_values.sum() - count_valid) / count_valid if count_valid > 0 else np.nan
    summary_list.append({'Combination': new_col, 'ROI': summary_value, 'Games': count_valid})
    final_df_unique[f"{new_col}_games"] = count_valid

# Create a summary DataFrame to display the summary values and game counts.
summary_df = pd.DataFrame(summary_list)
summary_df = summary_df.sort_values(by="ROI", ascending=False)
summary_df

,Combination,ROI,Games
65,HTXP_HT0.5UP,0.733727,495
46,FT12P_4.5OP,0.339364,39
84,BTTSP_2.5OP,0.334789,1637
76,HT12P_HT0.5OP,0.322260,136
60,HT1P_HT0.5OP,0.272484,1298
13,FTXP_2.5UP,0.258906,267
11,FTXP_1.5UP,0.238561,132
78,HT12P_HT1.5OP,0.234323,99
18,FTXP_BTTSP,0.199613,31
6,FT1P_4.5OP,0.184371,72
